In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Load dataset (replace 'news_articles.csv' with the path to your dataset)
data = pd.read_csv('news_articles.csv')

# Data preprocessing
# We'll use the 'text' and 'label' columns for this task
data = data[['text', 'label']].dropna()

# Splitting data into training and testing sets
X = data['text']
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Text transformation using TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Model training - Logistic Regression
model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, y_train)

# Model evaluation
y_pred = model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

# Saving the trained model and TF-IDF vectorizer for future use
joblib.dump(model, 'fake_news_detection_model.pkl')
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')

print("Model and vectorizer saved as 'fake_news_detection_model.pkl' and 'tfidf_vectorizer.pkl'.")


Accuracy: 0.6926829268292682
Classification Report:
              precision    recall  f1-score   support

        Fake       0.67      0.96      0.79       249
        Real       0.83      0.27      0.41       161

    accuracy                           0.69       410
   macro avg       0.75      0.62      0.60       410
weighted avg       0.73      0.69      0.64       410

Model and vectorizer saved as 'fake_news_detection_model.pkl' and 'tfidf_vectorizer.pkl'.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import joblib
import optuna
from optuna.samplers import TPESampler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

# Load dataset
data = pd.read_csv('news_articles.csv')

# Data preprocessing - select the 'text' and 'label' columns, drop any missing values
data = data[['text', 'label']].dropna()

# Splitting data into training and testing sets
X = data['text']
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model optimization process using Optuna
def objective(trial):
    # Hyperparameters for tuning
    C = trial.suggest_float("C", 1e-5, 10.0, log=True)
    max_df = trial.suggest_float("max_df", 0.5, 1.0)
    stop_words = 'english'
    
    # Create the pipeline: TF-IDF and Logistic Regression
    model_pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(stop_words=stop_words, max_df=max_df)),
        ('log_reg', LogisticRegression(C=C, max_iter=1000, random_state=42))
    ])

    # Perform cross-validation
    score = cross_val_score(model_pipeline, X_train, y_train, n_jobs=-1, cv=3, scoring="accuracy")
    return score.mean()

# Create Optuna study
sampler = TPESampler(seed=42)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=30)

# Best hyperparameters
best_params = study.best_params
print("Best hyperparameters:", best_params)

# Build the final model using the best hyperparameters
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=best_params['max_df'])
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Train the Logistic Regression model with optimized hyperparameters
model = LogisticRegression(C=best_params['C'], max_iter=1000, random_state=42)
model.fit(X_train_tfidf, y_train)

# Model evaluation
y_pred = model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

# Save the model and vectorizer
joblib.dump(model, 'optimized_fake_news_detection_model.pkl')
joblib.dump(tfidf_vectorizer, 'optimized_tfidf_vectorizer.pkl')

# Testing function to predict new examples
def test_model(examples):
    examples_tfidf = tfidf_vectorizer.transform(examples)
    predictions = model.predict(examples_tfidf)
    return predictions

# Example test over the dataset
test_examples = X_test[:5]
predicted_labels = test_model(test_examples)

# Display test results
test_results = pd.DataFrame({
    'Text': test_examples,
    'Predicted Label': predicted_labels,
    'Actual Label': y_test[:5].values
})

print(test_results)

# Save the test results
test_results.to_csv('test_results.csv', index=False)

print("Model and vectorizer saved as 'optimized_fake_news_detection_model.pkl' and 'optimized_tfidf_vectorizer.pkl'.")


In [18]:
import joblib
model = joblib.load('fake_news_detection_model.pkl')
tfidf_vectorizer = joblib.load('tfidf_vectorizer.pkl')

# To make predictions on new data
new_text = [" On September 15, President Joe Biden, Australian Prime Minister Scott Morrison, and British Prime Minister Boris Johnson jointly announced a new trilateral security partnership named AUKUS. The most significant part of the deal was the U.S. pledge to provide Australia with technology to build eight nuclear-powered (but not nuclear-armed) submarines"]
transformed_text = tfidf_vectorizer.transform(new_text)
prediction = model.predict(transformed_text)
prediction

array(['Fake'], dtype=object)